# EDA 

## Import Data Preparation Notebook

In [3]:
%run /Users/emmam/Documents/Flatiron/phase_2/film-analytics/code/data_preparation.ipynb

C:\Users\emmam
C:\Users\emmam\Documents
C:\Users\emmam\Documents\Flatiron
C:\Users\emmam\Documents\Flatiron\phase_2
C:\Users\emmam\Documents\Flatiron\phase_2\film-analytics
C:\Users\emmam\Documents\Flatiron\phase_2\film-analytics\data


In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.columns

In [ ]:
numeric_cols = df.select_dtypes(include=['number']).columns
df_numeric = df[numeric_cols]

Q1 = df_numeric.quantile(0.5)
Q3 = df_numeric.quantile(0.95)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df[~((df_numeric < lower_bound) | (df_numeric > upper_bound)).any(axis=1)]

## Descriptive Statistics

In [ ]:
df.describe()

## Correlation Analysis

In [ ]:
corr = df.corr()
corr

In [ ]:
corr_matrix = df[['vote_average', 'vote_count', 'worldwide_gross', 'popularity']].corr()
corr_matrix

## Visual Analysis

In [ ]:
genre_columns = ['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 
                'horror', 'music', 'mystery', 'romance', 'science fiction', 'thriller', 'war', 'western']

genre_gross = pd.DataFrame()

for genre in genre_columns:
    genre_gross.at[genre, 'average_worldwide_gross'] = df_filtered[df_filtered[genre] == 1]['worldwide_gross'].mean()

genre_gross = genre_gross.sort_values(by='average_worldwide_gross', ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x=genre_gross.index, y=genre_gross['average_worldwide_gross'])
plt.title('Average Worldwide Gross by Genre')
plt.ylabel('Average Worldwide Gross')
plt.xlabel('Genre')
plt.xticks(rotation=45)
plt.show();

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

director_gross = df_filtered.groupby('director')['worldwide_gross'].mean().sort_values(ascending=False)

#min_movies = 2
#director_gross = df_filtered.groupby('director').filter(lambda x: len(x) >= min_movies).groupby('director')['worldwide_gross'].mean().sort_values(ascending=False)

top_n = 20
top_directors = director_gross.head(top_n)

plt.figure(figsize=(12, 8))
top_directors.plot(kind='bar')
plt.title('Top Directors by Average Worldwide Gross')
plt.ylabel('Average Worldwide Gross')
plt.xlabel('Director')
plt.xticks(rotation=45)
plt.show();

In [ ]:
df_filtered['roi'] = ((df_filtered['worldwide_gross'] - df_filtered['production_budget']
                              ) /df_filtered['production_budget']) * 100

In [ ]:
df['worldwide_gross'].corr(df['animation'])

In [ ]:
genre_columns = ['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 
                 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 
                 'romance', 'science fiction', 'thriller', 'war', 'western']

genre_popularity_over_time = df_filtered.groupby('year')[genre_columns].mean()

plt.figure(figsize=(20, 15))
genre_popularity_over_time.plot(kind='line', lw=2)
plt.title('Genre Popularity Over Time')
plt.xlabel('Year')
plt.ylabel('Average Popularity')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.show();

In [ ]:
roi_min = df_filtered['worldwide_gross'].min()
roi_max = df_filtered['worldwide_gross'].max()
df_filtered['normalized_worldwide_gross'] = (df_filtered['worldwide_gross'] - roi_min) / (roi_max - roi_min)

vote_count_min = df_filtered['vote_count'].min()
vote_count_max = df_filtered['vote_count'].max()
df_filtered['normalized_vote_count'] = (df_filtered['vote_count'] - vote_count_min) / (vote_count_max - vote_count_min)

df_filtered['success_metric'] = (df_filtered['normalized_worldwide_gross'] + df_filtered['normalized_vote_count']) / 2

df_filtered.head()

In [ ]:
director_success = df_filtered.groupby('director')['success_metric'].mean().reset_index()

director_success = director_success.sort_values(by='success_metric', ascending=False).head(20)

plt.figure(figsize=(12, 6))
sns.barplot(x='director', y='success_metric', data=director_success, ci=None)
plt.xticks(rotation=45)
plt.title('Mean Success Metric by Director')
plt.xlabel('Director')
plt.ylabel('Mean Success Metric')
plt.show()


In [ ]:
melted_df = df_filtered.melt(id_vars=['primary_title', 'success_metric'], value_vars=genre_columns,
                    var_name='genre', value_name='genre_value')

melted_df = melted_df[melted_df['genre_value'] == 1]

plt.figure(figsize=(12, 6))
sns.boxplot(x='genre', y='success_metric', data=melted_df)
plt.xticks(rotation=45)
plt.title('Success Metric by Genre')
plt.xlabel('Genre')
plt.ylabel('Success Metric')
plt.show()

In [ ]:
melted_df = df_filtered.melt(id_vars=['primary_title', 'success_metric'], value_vars=genre_columns,
                    var_name='genre', value_name='genre_value')

melted_df = melted_df[melted_df['genre_value'] == 1]

plt.figure(figsize=(12, 6))
sns.boxplot(x='genre', y='success_metric', data=melted_df)
plt.xticks(rotation=45)
plt.title('Success Metric by Genre')
plt.xlabel('Genre')
plt.ylabel('Success Metric')
plt.show()

In [ ]:
df_filtered.head()

In [ ]:
df_filtered.columns

In [ ]:
df_filtered.head()

In [ ]:
# Convert 'release_date' to datetime
df_filtered = df_filtered.copy()

df_filtered.loc[:, 'release_date'] = pd.to_datetime(df_filtered['release_date'])

# Create 'release_year', 'release_month', and 'release_day' columns
df_filtered.loc[:, 'release_year'] = df_filtered['release_date'].dt.year
df_filtered.loc[:, 'release_month'] = df_filtered['release_date'].dt.month
df_filtered.loc[:, 'release_day'] = df_filtered['release_date'].dt.day

In [ ]:
df_filtered.columns

In [ ]:
def categorize_director(director, average_rating_per_director):
    high_rating_threshold = .5 
    low_rating_threshold = -.5   

    avg_rating = average_rating_per_director.get(director, None)

    if avg_rating is not None:
        if avg_rating >= high_rating_threshold:
            return 'high_rating'
        elif avg_rating <= low_rating_threshold:
            return 'low_rating'
        else:
            return 'medium_rating'
    else:
        return 'unknown'  

In [ ]:
average_rating_per_director = df_copy.groupby('director')['vote_average'].mean()

In [ ]:
df_copy['director_category'] = df_copy['director'].apply(lambda x: categorize_director(x, average_rating_per_director))

In [ ]:
df_copy['director_category'].value_counts()

In [ ]:
df_encoded1 = pd.get_dummies(df_copy, columns=['director_category'])

In [ ]:
df_encoded1.columns

In [ ]:
df_copy.columns

In [ ]:
df_encoded1 = pd.get_dummies(df_copy, columns=['studio'])

In [ ]:
df_encoded1.columns

In [ ]:
top_studios = df_copy['studio'].value_counts().nlargest(20).index

df_copy['studio_top'] = df_copy['studio'].apply(lambda x: x if x in top_studios else 'Other')

In [ ]:
df_encoded1.columns

In [ ]:
studio_df = pd.get_dummies(df_copy, columns=['studio_top'], drop_first=True)
studio_df.head()

In [ ]:
studio_df.columns[20:]

In [ ]:
df_encoded.columns

In [ ]:
# One-hot encode the categorical rating_category
df_encoded = pd.get_dummies(df_encoded, columns=['rating_category'], drop_first=True)

In [ ]:
df_encoded.columns

In [ ]:
columns_to_bring = ['release_day', 'release_month', 'release_year', 'title']
df_subset = df_filtered[columns_to_bring]

df_encoded = pd.merge(df_encoded, df_subset, on='title', how='left')

In [ ]:
df_encoded.head()